# Retry Query Engine

Uncomment to add your OpenAI API key or enable debugging

In [1]:
import os
# os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

First we ingest the Uber 10K

In [2]:
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index.query_engine.retry_source_query_engine import RetrySourceQueryEngine
from llama_index.readers.file.base import SimpleDirectoryReader


In [3]:
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()
index = VectorStoreIndex.from_documents(uber_docs)
query = "How many drivers does Uber have?"

We will query with the default settings first.

In [4]:
# Default query engine
default_query_engine = index.as_query_engine()
response = default_query_engine.query(query)
print(f"Default query engine response: {response}")

Default query engine response: 
It is not possible to answer this question with the given context information.


This didn't give us a good result, so let's try with retry.

In [5]:
from llama_index.query_engine.retry_query_engine import (
    RetryQueryEngine,
)
from llama_index.evaluation.base import QueryResponseEvaluator

# Query engine with retry
query_response_evaluator = QueryResponseEvaluator()
retry_query_engine = RetryQueryEngine(index.as_query_engine(), query_response_evaluator)
retry_response = retry_query_engine.query(query)
print(f"Query engine with retry response: {retry_response}")

Query engine with retry response: 
Here is a better answer.

It is not possible to determine the exact number of drivers Uber has with the given context information. However, it is known that Uber has more than 70,000 Mobility drivers in the UK and has extended offers to all eligible drivers who are not already represented by an attorney. Additionally, it is known that in October and November of 2016, outside actors downloaded the personal data of approximately 57 million Drivers and consumers worldwide.


Working better. The following are more experimental modules that aren't fully working at the moment.

The first tries to modify the source nodes by filtering the existing source nodes for the query based on llm node evaluation.

A better solution would be to re-retrieve based on filtering out the nodes that didn't pass evaluation.

TODO

In [6]:
retry_source_query_engine = RetrySourceQueryEngine(index.as_query_engine(), query_response_evaluator)
retry_source_response = retry_source_query_engine.query(query)
print(f"Query engine with retry source response: {retry_source_response}") 

Query engine with retry source response: 
It is not possible to answer this question with the given context information.


This module tries to use guidelines to direct the evaluator's behavior. You can customize your own guidelines.

Currently this can lead to the llm hallucinating. WIP


In [7]:
from llama_index.evaluation.guideline_eval import GuidelineEvaluator, DEFAULT_GUIDELINES
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.query_engine.retry_query_engine import (
    RetryGuidelineQueryEngine,
)

# Guideline eval
guideline_eval = GuidelineEvaluator(guidelines=DEFAULT_GUIDELINES) # just for example
typed_response = response if isinstance(response, Response) else response.get_response()
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")
feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")
retry_guideline_query_engine = RetryGuidelineQueryEngine(index.as_query_engine(), guideline_eval, resynthesize_query=True)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(f"Query engine with retry guideline response: {retry_guideline_response}")

Guideline eval evaluation result: The response should provide a specific number or statistic to answer the query. It should not be vague or ambiguous.
Transformed query: Here is a previous bad answer.

It is not possible to answer this question with the given context information.
Here is some feedback from the evaluator about the response given.
The response should provide a specific number or statistic to answer the query. It should not be vague or ambiguous.
Now answer the question.

What is the total number of Uber drivers?
Query engine with retry guideline response: 
According to Uber's 2020 Annual Report, there were 3.9 million active drivers on the Uber platform worldwide as of December 31, 2020.
